In [1]:
from pytrends.request import TrendReq
import pandas as pd
from datetime import datetime

In [2]:
pytrends = TrendReq(hl='en-US', tz=0) # 60 = CET 0 = UTC

# specifies current time as end date of results

now = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
now_d = int(now[:2])
now_m = int(now[3:5])
now_y = int(now[6:10])
now_h = int(now[11:13])

In [3]:
now

'25/11/2021 15:56:37'

In [4]:
kw_list = ['Ethereum']
coin_list = []
data = []

In [5]:
#data now
for kw in kw_list:
    coin_list = [kw]
    data.append(pytrends.get_historical_interest(
            coin_list, year_start=now_y-1, month_start=now_m, day_start=25,
            hour_start=now_h, year_end=now_y, month_end=now_m, day_end=24,
            hour_end=now_h, cat=0, sleep=0))
data

[                     Ethereum isPartial
 date                                   
 2020-11-25 15:00:00        73     False
 2020-11-25 16:00:00       100     False
 2020-11-25 17:00:00        73     False
 2020-11-25 18:00:00        71     False
 2020-11-25 19:00:00        74     False
 ...                       ...       ...
 2021-11-24 12:00:00        60     False
 2021-11-24 13:00:00        62     False
 2021-11-24 14:00:00        66     False
 2021-11-24 15:00:00        66     False
 2021-11-24 15:00:00        98     False
 
 [7437 rows x 2 columns]]

In [6]:
df = data[0].iloc[:,0]

In [7]:
df = pd.DataFrame(df) 
df = df.reset_index()
df.head(5)

,date,Ethereum
0,2020-11-25 15:00:00,73
1,2020-11-25 16:00:00,100
2,2020-11-25 17:00:00,73
3,2020-11-25 18:00:00,71
4,2020-11-25 19:00:00,74


In [8]:
df['hour'] = df['date'].astype(str)
df['hour'] = df['hour'].str.slice(11, 13, 1)
df = df[df['hour'] == '00']
df

,date,Ethereum,hour
9,2020-11-26,67,00
33,2020-11-27,61,00
57,2020-11-28,50,00
81,2020-11-29,50,00
105,2020-11-30,68,00
...,...,...,...
7324,2021-11-20,71,00
7348,2021-11-21,67,00
7372,2021-11-22,59,00
7396,2021-11-23,66,00


In [9]:
df.to_csv('eth_daily_google_trends_1y.csv')